https://github.com/OpenBMB/MiniCPM-o \
https://huggingface.co/openbmb/MiniCPM-V-2_6

In [2]:
import os
import sys

In [3]:
# Set CUDA_VISIBLE_DEVICES to expose only device 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Set CUDA_VISIBLE_DEVICES to expose devices 0, 1, 2, and 3
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [4]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

/data/mn27889/miniconda3/envs/path-opendata/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True,
                                   attn_implementation='sdpa', torch_dtype=torch.bfloat16) # sdpa or flash_attention_2, no eager

model = model.cuda().eval()

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 23.90it/s]


In [8]:
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6:
- tokenization_minicpmv_fast.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [7]:
image = Image.open('./DeepSeek-VL2/images/visual_grounding_1.jpeg').convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': [image, question]}]

In [9]:
res = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer
)

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6:
- image_processing_minicpmv.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2_6:
- processing_minicpmv.py
- image_processing_minicpmv.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/data/mn27889/miniconda3/envs/path-opendata/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:640: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with

In [10]:
print(res)

The image features two giraffes standing in a grassy field with trees in the background.


#### Multiple Images

In [11]:
image1 = Image.open('./DeepSeek-VL2/images/visual_grounding_1.jpeg').convert('RGB')
image2 = Image.open('./DeepSeek-VL2/images/visual_grounding_2.jpg').convert('RGB')
question = 'Compare image 1 and image 2, tell me about the differences between image 1 and image 2.'

msgs = [{'role': 'user', 'content': [image1, image2, question]}]

In [12]:
answer = model.chat(
    image=None,
    msgs=msgs,
    tokenizer=tokenizer
)

In [13]:
print(answer)

The main differences between the two images are their subjects and contexts. In image 1, there is a natural setting with giraffes, suggesting a wildlife or safari environment. On the other hand, image 2 depicts an animated scene of people in business attire, indicating a more urban or professional context. The emotions conveyed also differ, as the first image shows calm and serene animals, while the second one portrays a heated interaction between characters.
